In [1]:
import re
from nltk.tokenize import TweetTokenizer
import pandas as pd
from ast import literal_eval
import numpy as np
import os

In [81]:
def add_wk_to_tweets():
    filenames = os.listdir('../team_data/')
    for file in filenames[:1]:
        print(file)
        wk = file[3:5]
        print(wk)
        df = pd.read_csv(f'../team_data/{file}')
        df.insert(0, 'week', wk)
        df.to_csv(f'../team_data_wk/{file}')


def get_all_tweets():
    filenames = os.listdir(f'../team_data_wk')
    df_all = pd.concat([
                pd.read_csv(
                    f'../team_data_wk/{f}',
                    converters={"tweet_text":lambda x:literal_eval(x).decode("utf8")}
                ) for f in filenames
            ], ignore_index=True)
    df_all = df_all.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
    return df_all
        

In [2]:
df_all_tweets = pd.read_csv('../team_data/all_tweets.csv')


In [82]:
# df_all_tweets = get_all_tweets()

In [9]:
df_all_tweets
# df_all_tweets.to_csv('../team_data/all_tweets.csv')

,week,team,timestamp,tweet_text,userid
0,12,49ers,2022-11-27 20:24:43+00:00,1hr til kick off. Let’s Go Niners!! #fttb ❤️💛,1404574279811878913
1,12,49ers,2022-11-27 20:24:42+00:00,Big test today for Jimmy Garoppolo and the #49...,38617573
2,12,49ers,2022-11-27 20:24:03+00:00,#49ers have held three straight opponents scor...,38617573
3,12,49ers,2022-11-27 20:23:47+00:00,@49erHodge Hell yea GO NINERS 🤘 #FTTB,601772815
4,12,49ers,2022-11-27 20:23:38+00:00,#49ers are seeking their first 4-game regular-...,38617573
...,...,...,...,...,...
645057,19,vikings,2023-01-11 20:33:58+00:00,@heykayadams @UpAndAdamsShow @kgxix @Young_Sla...,1422946127750606853
645058,19,vikings,2023-01-11 20:32:47+00:00,Here’s the first #Giants injury report for the...,1561574120
645059,19,vikings,2023-01-11 20:32:00+00:00,22 Things the 2022 Regular Season Revealed Abo...,3035196396
645060,19,vikings,2023-01-11 20:31:00+00:00,Cook on The Running Game Heading Into Sunday: ...,3035196396


In [8]:
df_all_tweets = df_all_tweets.drop(['Unnamed: 0'], axis=1)

In [13]:
df_some_tweets = df_all_tweets.iloc[:10000]
df_some_tweets

,week,team,timestamp,tweet_text,userid
0,12,49ers,2022-11-27 20:24:43+00:00,1hr til kick off. Let’s Go Niners!! #fttb ❤️💛,1404574279811878913
1,12,49ers,2022-11-27 20:24:42+00:00,Big test today for Jimmy Garoppolo and the #49...,38617573
2,12,49ers,2022-11-27 20:24:03+00:00,#49ers have held three straight opponents scor...,38617573
3,12,49ers,2022-11-27 20:23:47+00:00,@49erHodge Hell yea GO NINERS 🤘 #FTTB,601772815
4,12,49ers,2022-11-27 20:23:38+00:00,#49ers are seeking their first 4-game regular-...,38617573
...,...,...,...,...,...
9995,12,browns,2022-11-26 05:42:04+00:00,“Things I Think I know about the Cleveland #Br...,1488755185
9996,12,browns,2022-11-26 05:02:34+00:00,Mac for Mutts training camp? #browns https://t...,1021217082803974144
9997,12,browns,2022-11-26 05:01:21+00:00,Just watching Myles’s press conference going i...,913342800
9998,12,browns,2022-11-26 04:57:33+00:00,Odell Beckham Jr. 1⃣3⃣ Career Stats: -531 R...,4891213336


## TWEET NLP VERSION

In [10]:
import tweetnlp

In [11]:
model_nlp = tweetnlp.load_model('sentiment')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 18 seconds for 100 tweets

In [12]:
def get_sentiment_nlp(df, model):
    # convert tweets to list for model
    data = df['tweet_text'].to_list()
    # conduct sentiment analysis with model
    sentiment = model.sentiment(data, return_probability=True, )
    # create dataframe with sentiment using json normalize to flatten structure
    sentiment_df = pd.json_normalize(sentiment)
    # concat sentiment df with tweet df and return
    df_tweets_w_sentiment = pd.concat([df, sentiment_df], axis=1)
    return df_tweets_w_sentiment

In [14]:
df_sentiment_nlp = get_sentiment_nlp(df_some_tweets, model_nlp)


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3932160000 bytes.

In [122]:
df_sentiment_nlp.head()

,week,team,timestamp,tweet_text,userid,label,probability.negative,probability.neutral,probability.positive
0,12,49ers,2022-11-27 20:24:43+00:00,1hr til kick off. Let’s Go Niners!! #fttb ❤️💛,1404574279811878913,positive,0.001576,0.014757,0.983667
1,12,49ers,2022-11-27 20:24:42+00:00,Big test today for Jimmy Garoppolo and the #49...,38617573,neutral,0.011927,0.865918,0.122156
2,12,49ers,2022-11-27 20:24:03+00:00,#49ers have held three straight opponents scor...,38617573,neutral,0.004039,0.501523,0.494438
3,12,49ers,2022-11-27 20:23:47+00:00,@49erHodge Hell yea GO NINERS 🤘 #FTTB,601772815,positive,0.003349,0.057407,0.939245
4,12,49ers,2022-11-27 20:23:38+00:00,#49ers are seeking their first 4-game regular-...,38617573,neutral,0.003485,0.511220,0.485296


## TRANSFORMER CLASSIFICATION EXAMPLE
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest


In [23]:
from transformers import pipeline
model = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
sentiment_pipeline = pipeline(model=model)


loading configuration file https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest/resolve/main/config.json from cache at C:\Users\hugho/.cache\huggingface\transformers\c26252806565e705085b65f69d7d544c05112fee06744845d6c067efcb278fff.31fdd4298ba667598119e493f82afb18fcd41f96366700ec7d6460c17c421feb
Model config RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-sentiment-latest",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "robe

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

loading configuration file https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest/resolve/main/config.json from cache at C:\Users\hugho/.cache\huggingface\transformers\c26252806565e705085b65f69d7d544c05112fee06744845d6c067efcb278fff.31fdd4298ba667598119e493f82afb18fcd41f96366700ec7d6460c17c421feb
Model config RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-sentiment-latest",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "robe

1000 tweets = 100 seconds = 00:01:40 min
100,000 tweets => 10000 seconds = 2:45:00 

In [96]:

print(645000/10/60/60, 'hours for 645,000 tweets')

17.916666666666668 hours for 645,000 tweets


In [103]:
def get_sentiment(df, pipeline):
    # convert tweets to list for pipeline
    data = df['tweet_text'].to_list()
    # conduct sentiment analysis through pipeline
    sentiment = sentiment_pipeline(data)
    # create dataframe with sentiment
    sentiment_df = pd.DataFrame.from_dict(sentiment)
    # concat sentiment df with tweet df and return
    df_tweets_w_sentiment = pd.concat([df, sentiment_df], axis=1)
    return df_tweets_w_sentiment


In [124]:
df_some_sentiment = get_sentiment(df_some_tweets, sentiment_pipeline)

In [106]:
df_some_sentiment.head()

,week,team,timestamp,tweet_text,userid,label,score
0,12,49ers,2022-11-27 20:24:43+00:00,1hr til kick off. Let’s Go Niners!! #fttb ❤️💛,1404574279811878913,positive,0.983667
1,12,49ers,2022-11-27 20:24:42+00:00,Big test today for Jimmy Garoppolo and the #49...,38617573,neutral,0.865917
2,12,49ers,2022-11-27 20:24:03+00:00,#49ers have held three straight opponents scor...,38617573,neutral,0.501522
3,12,49ers,2022-11-27 20:23:47+00:00,@49erHodge Hell yea GO NINERS 🤘 #FTTB,601772815,positive,0.963333
4,12,49ers,2022-11-27 20:23:38+00:00,#49ers are seeking their first 4-game regular-...,38617573,neutral,0.511220


## Fine Tune Train model
would require labeling some data - not going to do I don't think

In [17]:
language_model = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
task = "sentiment"

# load dataset
dataset, label_to_id = tweetnlp.load_dataset(task)

# load trainer
trainer_class = tweetnlp.load_trainer(task)

# define trainer
trainer = trainer_class(
    language_model=language_model,
    dataset=dataset,
    label_to_id=label_to_id,
    max_length=128,
    split_train='train',
    split_test='test',
    output_dir=f'model_ckpt/test'
)

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to C:/Users/hugho/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


...

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

c:\Users\hugho\AppData\Local\Programs\Python\Python310\lib\site-packages\tweetnlp\text_classification\trainer.py:94: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_accuracy = load_metric("accuracy")


In [108]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45615
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12284
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [18]:
# train
trainer.train(down_sample_size_train=1000, ray_result_dir="ray_results/test")

# save model checkpoint
trainer.save_model()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\hugho\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 45615
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 17106


  0%|          | 0/17106 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [6]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)



Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# scores

In [10]:
ranking = np.argsort(scores)
ranking = ranking[::-1]

In [11]:


# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores

for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


1) negative 0.7236
2) neutral 0.2287
3) positive 0.0477
